# HDA - Project 3

In [1]:
import utils
import deeplearning
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns 

from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

from keras import regularizers
from keras.activations import relu
from keras.layers import Conv2D, BatchNormalization, Dropout, LeakyReLU, Flatten, Activation, Dense, MaxPooling2D, LSTM, Reshape
from keras.models import load_model, Model, Sequential
from keras.optimizers import Adam

Using TensorFlow backend.


The following cell contains the hyper-parameters that can be tuned for code execution:
- subject: select the subject on which to test the model, between [1,4];
- folder: directory name where '.mat' files are stored;
- label_col: column of features to be selected to perform activity detection, between [0,6]:

|  Label |  Feature |
|:-:     |:-:|
|  0     | Locomotion (TASK A)  |
|  1     | High Level Activity |
|  2     | Low Level Left Arm  |
|  3     | Low Level Left Arm Object  |
|  4     | Low Level Right Arm  |
|  5     | Low Level Right Arm Object  |
|  6     | Medium Level Both Arms (TASK B2) |

- window_size: parameter that sets the length of temporal windows on which to perform the convolution;
- stride: step length to chose the next window.

The size of the temporal window seems to be fundamental in order to get a more specific and powerful model; of course the choice of the step lenght between consequent windows has to be consistent and to make sense. Thinking about a real-time situation, as long as we collect data we can use a sliding window of real-time samples; in this way, it is reasonable to use also a small value for the stride. Another important reason behind the choice of the value of the 

In [2]:
subjects = [1,2,3,4]
folder = "./data/full/"
#folder = "/floyd/input/hdadataset/full/" # To be used with FloydHub
label = 0     # default for task A
window_size = 64
stride = 3

# Classification

After the _detection_ step, this time we exclude all the samples associated to the _null class_; in this way we can build a neural network cleaned of the null class and that can distinguish better the difference between motions.

### Model definition, compilation and input reshaping

In [3]:
n_features = 110 #number of features taken into consideration for the solution of the problem
n_classes = 4

In [11]:
classification_model = deeplearning.MotionClassification((window_size,n_features,1), n_classes)
classification_model.summary() # model visualization

classification_model.compile(optimizer = Adam(lr=0.01), 
                   loss = "categorical_crossentropy", 
                   metrics = ["accuracy"])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_7 (Batch (None, 64, 110, 1)        4         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 54, 110, 50)       600       
_________________________________________________________________
batch_normalization_8 (Batch (None, 54, 110, 50)       200       
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 44, 110, 25)       13775     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 22, 110, 25)       0         
_________________________________________________________________
batch_normalization_9 (Batch (None, 22, 110, 25)       100       
_________________________________________________________________
reshape_4 (Reshape)          (None, 22, 2750)          0         
__________

### Model fitting

After the training procedure, the model will be saved on disk

In [ ]:
for s in subjects:
    
    print("Going for USER ", s)
    
    [x_train, y_train, x_test, y_test, n_classes] = utils.preprocessing(s,
                                                                    folder,
                                                                    label,
                                                                    window_size,
                                                                    stride,
                                                                    null_class = False)
    
    input_train = x_train.reshape(x_train.shape[0], window_size, n_features, 1)
    input_test = x_test.reshape(x_test.shape[0], window_size, n_features, 1)
    
    classification_model.fit(x = input_train, 
                   y = y_train, 
                   epochs = 20, 
                   batch_size = 300,
                   verbose = 1,
                   validation_data=(input_test, y_test))

classification_model.save('./classification_model_A.h5')
classification_model.save_weights('./classification_model_weights_A.h5')

In [17]:
#classification_model = load_model('./data/classification_model.h5')

output_test = classification_model.predict(input_test)
prediction = np.argmax(output_test, axis=1)

print("Accuracy: ", accuracy_score(np.argmax(y_test, axis=1), prediction))
print("F1-measure: ", utils.f1_score(np.argmax(y_test, axis=1), prediction, average='weighted'))

cnf_matrix = utils.confusion_matrix(np.argmax(y_test, axis=1), prediction)
np.set_printoptions(precision=2)

sns.set_style("dark")
plt.figure()
utils.plot_confusion_matrix(cnf_matrix, classes=[1,2,4,5],
                      title='Confusion matrix, without normalization')